In [12]:
# Libraries we need to install - If it is already installed you can skip this cell
!pip install sentencepiece
!pip install transformers

In [13]:
# Libraries 

# base model
from torchtext.legacy.data import Field,LabelField,BucketIterator,TabularDataset
from torchtext import vocab
from tqdm import tqdm
import torch.nn.functional as F

#preprocessing and evaluation
import pandas as pd
import numpy as np
import os
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

In [15]:
# Parameters

seed = 1234
np.random.seed(seed) 
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True  # cuda algorithms
os.environ['PYTHONHASHSEED'] = str(seed)
    
batch_size = 32
lr = 1e-4
num_epochs = 500
model_name = 'lstm' # model name (bert, alberta, distilbert or gpt2  for pretrained) (lstm, rnn, bilestm for base model)
output_path = "output-bert" #create a folder to save pretrained model
model_path = "lstm"
embedding_path = "embeddings/glove.6B.50d.txt"
max_length = 512
dataset_base = True # boolean value to split dataset into 
dataset_path = "data/" # path where to save splitted data (it is necessary is dataset_base is True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # use 'cuda' if available else 'cpu'

In [16]:
#Utils function 
def split_dataset_base(dataframe):
    """ Split dataset into train, val and test
    Input:
        dataframe - dataframe dataset
    Returns:
        train_df dataframe train dataframe
        val_df dataframe val dataframe
        test_df dataframe test dataframe
    """   
    # split train dataset into train, validation and test sets
    df, test_df = train_test_split(dataframe,random_state=seed,test_size=0.2, stratify=dataframe["label_encoded"])
    
    train_df, val_df = train_test_split(df,random_state=seed,test_size=0.2, stratify=df["label_encoded"])
    
    save_files(train_df, val_df, test_df)
    
    return train_df, val_df, test_df

def save_files(out_path, train_df, val_df, test_df):
    """ Save splittted dataset into folder
    Input:
        out_path string path for saving the files
        train_df dataframe train dataframe
        val_df dataframe val dataframe
        test_df dataframe test dataframe
    """  
    train_df.to_csv(out_path+'train.csv',index=False)
    val_df.to_csv(out_path+'val.csv',index=False)
    test_df.to_csv(out_path+'test.csv',index=False) 
   
def tokenize(s): 
    """ Split text
    Input:
        s string text to split
    Returns:
        string splittex text
    """  
    return s.split(' ')

def compute_metrics(p):
    """Compute metrics for evaluation
    p Lists prediction and gold labels for evaluation
    Reurns:
        eval_scores dictionary evaluation scores
    """
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    eval_scores = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
    
    return eval_scores

In [21]:
# Network for Base models
class Network(torch.nn.Module):
    '''
    It inherits the functionality of Module class from torch.nn whic includes al the layers, weights, grads setup
    and methods to calculate the same. We just need to put in the required layers and describe the flows as
    which layers comes after which one
    '''
    
    def __init__(self,in_neuron,embedding_dim=50,hidden_size=256,out_neuron=8,m_type='lstm',drop=0.2,**kwargs):
        '''
        Constructor of the class which will instantiate the layers while initialisation.
        
        Input:
            in_neuron: input dimensions of the first layer {int}
            embedding_dim: number of latent features you want to calculate from the input data {int} default=128
            hidden_size: neurons you want to have in your hidden RNN layer {int} default=256
            out_neuron: number of outputs you want to have at the end.{int} default=1
            model: whether to use 'rnn' or 'lstm' {string} 
            drop: proportion of values to dropout from the previous values randomly {float 0-1} default=0.53
            **kwargs: any torch.nn.RNN or torch.nn.LSTM args given m_type='rnn' or'lstm' {dict}
        Returns: 
            A tensor of shape {batch,out_neuron} as output 
        '''
        super(Network,self).__init__() 
        self.m_type = m_type
        
        self.embedding = torch.nn.Embedding(in_neuron,embedding_dim) # embedding layer is always the first layer
        if self.m_type == "bilstm":
            self.bilstm = torch.nn.LSTM(embedding_dim,hidden_size,bidirectional=True, **kwargs)
        elif self.m_type == 'lstm':
        # whether to use the LSTM type model or the RNN type model. It'll use only 1 in forward()
            self.lstm = torch.nn.LSTM(embedding_dim,hidden_size,**kwargs)
        else:
            self.rnn = torch.nn.RNN(embedding_dim,hidden_size,**kwargs) 
        
        self.dropout = torch.nn.Dropout(drop) # drop the values by random which comes from previous layer
        if self.m_type == "bilstm":
            self.dense = torch.nn.Linear(hidden_size*2,out_neuron) # last fully connected layer
        else:
            self.dense = torch.nn.Linear(hidden_size,out_neuron) # last fully connected layer
    
    def forward(self,t):
        '''
        Activate the forward propagation of a batch at a time to transform the input bath of tensors through
        the different layers to get an out which then will be compared to original label for computing loss.
        Input:
            t: tensors in the form of a batch {torch.tensor}
        Returns:
            output of the network
        '''
        embedding_t = self.embedding(t)
        
        drop_emb = self.dropout(embedding_t)
        
        if self.m_type == "bilstm":
            out, (hidden_state,_) = self.bilstm(drop_emb)
            hidden_state = torch.cat((hidden_state[0,:,:],hidden_state[1,:,:]), dim=1)
        elif self.m_type == 'lstm':
            out, (hidden_state,_) = self.lstm(drop_emb)
        else:
            out, hidden_state = self.rnn(drop_emb)
            #  shape of rnn_out = (seq_len, batch, num_directions * hidden_size)
       
        hidden_squeezed = hidden_state.squeeze(0) 
        
        return self.dense(hidden_squeezed)
    
def prepare_dataset_base(dataset, train):
    """ Prepare dataset
    Input:
        dataset - dataframe 
    Returns:
        train_dataset - BucketIterator train Dataset
        val_dataset - BucketIterator val Dataset
        test_dataset - BucketIteratortest Dataset
        input_size int input size of the model
    """
    
    text_field = Field(tokenize=tokenize)
    label_field = LabelField(dtype=torch.float) 
    # useful for label string to LabelEncoding. Not useful here but doesn't hurt either
    
    fields = [('sentences',text_field),('label_encoded',label_field)] 
    # (column name,field object to use on that column) pair for the dictonary
    
    glove = vocab.Vectors(embedding_path, dataset_path)
    if train: #prepare train val and est dataset

        if not dataset_base: #ıf dataset is not saved
            train_df, val_df, test_df = split_dataset_base(dataset)
            
        train_dataset, val_dataset, test_dataset = TabularDataset.splits(path=dataset_path, train='train.csv',validation='val.csv',test='test.csv', 
                                                 format='csv',skip_header=True,fields=fields)
        
        
        
        text_field.build_vocab(train_dataset,max_size=100000,vectors=glove,unk_init=torch.Tensor.zero_) 
        label_field.build_vocab(train_dataset) 
        input_size = len(text_field.vocab)
        train_iter, val_iter, test_iter = BucketIterator.splits((train_dataset, val_dataset, test_dataset), batch_sizes=(32,128,128),
                                                      sort_key=lambda x: len(x.sentences),
                                                      sort_within_batch=False,
                                                      device=device) # use the cuda device if available
        return train_iter, val_iter, test_iter, input_size
    else: #prepare dataset for tes
        test_dataset = TabularDataset(path=dataset_path+'test.csv', 
                                             format='csv',skip_header=True,fields=fields)
        
        text_field.build_vocab(test_dataset,max_size=100000,vectors=glove,unk_init=torch.Tensor.zero_) 
        label_field.build_vocab(test_dataset)
        
        test_iter = BucketIterator(test_dataset, batch_size=32,
                                                      sort_key=lambda x: len(x.sentences),
                                                      sort=False,
                                                      sort_within_batch=False,
                                                      device=device) # use the cuda device if available
    
        return test_dataset

In [23]:
# save and load models for base models
def save_checkpoint(save_path, model_name, optimizer, valid_loss, in_neuron):

    if save_path == None:
        return
    
    state_dict = {'model_state_dict': model.state_dict(),
                  'optimizer_state_dict': optimizer.state_dict(),
                  'valid_loss': valid_loss,
                 'input_size':in_neuron}
    
    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_checkpoint(load_path, model_name):

    if load_path==None:
        return
    
    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')
    
    model = Network(state_dict['input_size'], m_type=model_name) 
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) 
    model.load_state_dict(state_dict['model_state_dict'])
    optimizer.load_state_dict(state_dict['optimizer_state_dict'])
    
    return model, optimizer

In [24]:
# Read dataset
def read_dataset(file_path):
    """ Read dataset
    Input:
        file_path - string the path of the dataset
    Returns:
        train dataframe 
    """
    train_data = pd.read_excel(file_path, 'Sheet1')
    
    ''' Should/Must statement
        Should/must statement
        should/must statement labels are 
        converted to Should/Must statement
        
        personalizing is converted to Personalizing''' 
    
    
    train_data.loc[(train_data['label'] == 'should/must statement') | (train_data['label'] == 'Should/must statement')] = 'Should/Must statement' 
    train_data.loc[train_data['label'] == 'personalizing'] = 'Personalizing' 
    
    #Label encoding 
    
    le = LabelEncoder()
    train_data["label_encoded"] = le.fit_transform(train_data["label"]) 
    np.save('classes.npy', le.classes_)
    return train_data

In [18]:
# prepare dataset for pretrained models
#split into train val and test
dataset = read_dataset('L2400.xlsx')

num_output = len(set(dataset["label_encoded"])) # number of classes in the dataset

In [25]:
#train function for base models
def train_network(network,train_iter,optimizer,loss_fn,epoch_num):
    '''
    train the network using given parameters
    Input:
        network: any Neural Network object 
        train_batch: iterator of training data
        optimizer: optimizer for gradients calculation and updation
        loss_fn: appropriate loss function
        epoch_num = Epoch number so that it can show which epoch number in tqdm Bar
    Returns:
        a tuple of (average_loss,average_accuracy) of floating values for a single epoch
    '''
    epoch_loss = 0
    epoch_acc = 0 
    network.train() 
    
    for batch in tqdm(train_iter,f"Epoch: {epoch_num}"): 
        optimizer.zero_grad() 
        predictions = network(batch.sentences).squeeze(1) 
        loss = loss_fn(predictions,batch.label_encoded.to(torch.long)) 
        pred_classes = F.softmax(predictions, dim=1)
        pred_classes = torch.argmax(pred_classes, dim=1)
        correct_preds = (pred_classes == batch.label_encoded).float()
        accuracy = correct_preds.sum()/len(correct_preds)# it'll be a tensor of shape [1,]
        loss.backward() 
        optimizer.step()
        
        epoch_loss += loss.item() 
        epoch_acc += accuracy.item()
        
        
    return epoch_loss/len(train_iter), epoch_acc/len(train_iter)

In [26]:
#evaluation function for base models
def evaluate_network(network,val_test_iter,optimizer,loss_fn):
    '''
    evaluate the network using given parameters
    args:
        network: any Neural Network object 
        val_test_iter: iterator of validation/test data
        optimizer: optimizer for gradients calculation and updation
        loss_fn: appropriate loss function
    out:
        a tuple of (average_loss,average_accuracy) of floating values for the incoming dataset
    '''
    total_loss = 0 
    total_acc = 0
    network.eval()
    
    with torch.no_grad():
        
        for batch in val_test_iter:

            predictions = network(batch.sentences).squeeze(1)
            loss = loss_fn(predictions,batch.label_encoded.to(torch.long))
            pred_classes = torch.argmax(predictions, dim=1)
            correct_preds = (pred_classes == batch.label_encoded).float()
            accuracy = correct_preds.sum()/len(correct_preds)
            total_loss += loss.item() 
            total_acc += accuracy.item()

        return total_loss/len(val_test_iter), total_acc/len(val_test_iter)

In [27]:
# Train base model
train_iter, val_iter, test_iter, in_neuron  = prepare_dataset_base(dataset, True)

network = Network(in_neuron, m_type=model_name) 
if torch.cuda.is_available():
    network.cuda() 

optimizer = torch.optim.Adam(network.parameters(),lr=lr) 
loss_fn = torch.nn.CrossEntropyLoss()

for epoch in range(num_epochs):
        train_loss, train_acc = train_network(network,train_iter,optimizer,loss_fn,epoch+1)
        val_loss,val_acc = evaluate_network(network,val_iter,optimizer,loss_fn)
        tqdm.write(f'''End of Epoch: {epoch+1}  |  Train Loss: {train_loss:.3f}  |  Val Loss: {val_loss:.3f}  |  Train Acc: {train_acc*100:.2f}%  |  Val Acc: {val_acc*100:.2f}%''')
        
test_loss,test_acc = evaluate_network(network,test_iter,optimizer,loss_fn)
save_checkpoint(output_path + '/model.pt', network, optimizer, val_loss, in_neuron)

Epoch: 1: 100%|█████████████████████████████████| 48/48 [00:01<00:00, 45.21it/s]


End of Epoch: 1  |  Train Loss: 2.081  |  Val Loss: 2.085  |  Train Acc: 12.37%  |  Val Acc: 13.80%


Epoch: 2: 100%|█████████████████████████████████| 48/48 [00:00<00:00, 61.32it/s]


End of Epoch: 2  |  Train Loss: 2.079  |  Val Loss: 2.088  |  Train Acc: 11.52%  |  Val Acc: 13.54%


Epoch: 3: 100%|█████████████████████████████████| 48/48 [00:00<00:00, 63.39it/s]


...

End of Epoch: 75  |  Train Loss: 0.661  |  Val Loss: 1.417  |  Train Acc: 78.12%  |  Val Acc: 54.17%


Epoch: 76: 100%|████████████████████████████████| 48/48 [00:00<00:00, 60.04it/s]


End of Epoch: 76  |  Train Loss: 0.690  |  Val Loss: 1.435  |  Train Acc: 76.69%  |  Val Acc: 51.30%


Epoch: 77:  48%|███████████████▎                | 23/48 [00:00<00:00, 59.02it/s]


KeyboardInterrupt: 

In [ ]:
# Test base model without training
dataset = read_dataset('L2400.xlsx')
test_data_file = "path of the file"
#dataset = read_dataset(test_data_file)

test_iter  = prepare_dataset_base(dataset, False)


model, optimizer = load_checkpoint(model_path + '/model.pt', model_name)



#network.load_state_dict(torch.load(model_path), strict=False)
test_loss,test_acc = evaluate_network(model,test_iter,optimizer,loss_fn)